In [ ]:
#importation des bibliothèques
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import time
import random

In [47]:
matrice_proba = np.loadtxt('transitions_alphabets.txt')

In [48]:
#verification
print(matrice_proba)

[[0.00000000e+00 9.09075846e-02 1.48907496e-02 7.03168151e-02
  1.16115812e-01 9.71046992e-02 2.63294523e-02 9.97503480e-03
  7.25206018e-03 3.42829684e-02 2.08724565e-02 4.97094757e-05
  9.04933389e-02 5.34542562e-02 2.92346950e-02 2.15407728e-02
  8.13744118e-02 5.32222787e-02 2.69204427e-02 6.96982082e-02
  2.81742262e-02 2.65890462e-02 2.69977686e-02 6.07560259e-05
  1.10465502e-04 3.88838566e-03 1.43605152e-04]
 [1.55572450e-01 1.46200968e-05 1.57750844e-02 2.29096917e-02
  8.07029343e-03 1.75441161e-04 2.35383558e-03 1.78218980e-02
  1.65207094e-03 2.61348850e-01 1.69593123e-03 1.46200968e-05
  3.35531221e-02 2.45471425e-02 1.46654191e-01 3.07022032e-04
  1.98540914e-02 3.88894574e-03 7.20770772e-02 4.48252167e-02
  3.29390781e-02 6.50594307e-02 6.22523721e-02 0.00000000e+00
  1.02340677e-04 6.12582055e-03 4.09362710e-04]
 [1.93143409e-03 8.32126187e-02 8.04764204e-04 0.00000000e+00
  1.28762273e-03 1.51939482e-01 0.00000000e+00 0.00000000e+00
  1.60952841e-04 1.69805247e-01 8.69

In [6]:
def calcul_V(chaine):
    V = 0
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"  
    
    for i in range(len(chaine) - 1):
        char1 = chaine[i]
        char2 = chaine[i + 1]
        
        # Gestion des indices
        if char1 == " ":
            indice1 = 0
        elif char1 in alphabet:
            indice1 = alphabet.index(char1) + 1
        else:
            continue  # Ignorer les caractères non reconnus
        
        if char2 == " ":
            indice2 = 0
        elif char2 in alphabet:
            indice2 = alphabet.index(char2) + 1
        else:
            continue  # Ignorer les caractères non reconnus
        
        # Calcul de V en utilisant les probabilités de la matrice
        V += np.log(matrice_proba[indice1][indice2] + 1e-15)
    
    # Moyenne normalisée par la longueur de la chaîne
    if len(chaine) > 1:
        V /= len(chaine)
    return V



In [7]:
# Exemple d'utilisation
ma_chaine = "BOUNJOUR TOUT LE MONDE"
resultats = calcul_V(ma_chaine)

print(resultats)

-2.0571527039988253


In [8]:
# permutation de 2 caractères choisit de manière aleatoire
def generer_alea(texte):
    code_voisin = texte[:]
    i, j = npr.choice(range(len(texte)), size = 2,replace=False) 
    code_voisin[i], code_voisin[j] = code_voisin[j], code_voisin[i] 
    return code_voisin

In [26]:
# Algorithme de recuit simulé pour déchiffrer un code secret avec permutations
def recuit_sim_dechiffrement(chaine_init, iterations, h):
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ "
    chaine_actuelle = chaine_init
    meilleure_chaine = chaine_init
    meilleur_V = calcul_V(chaine_init)
    chaine_actuelle_V = meilleur_V

    for i in range(iterations):
        # Générer une chaîne de caractères voisine par permutation de deux caractères aléatoires
        idx1 = npr.randint(len(chaine_init))
        idx2 = npr.randint(len(chaine_init))
        voisin = list(chaine_actuelle)
        voisin[idx1], voisin[idx2] = voisin[idx2], voisin[idx1]
        voisin = ''.join(voisin)
        
        # Calcul de la variation de potentiel
        delta_V = calcul_V(voisin) - chaine_actuelle_V
        
        # Température décroissante
        T = h / np.log(i + 2)
        alpha = min(1, np.exp(-delta_V / T))
        
        # Décision d'acceptation du voisin
        if npr.rand() <= alpha:
            chaine_actuelle = voisin
            chaine_actuelle_V = calcul_V(chaine_actuelle)
            if chaine_actuelle_V > meilleur_V:
                meilleure_chaine = chaine_actuelle
                meilleur_V = chaine_actuelle_V
        
    return meilleure_chaine

In [ ]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import random
from collections import Counter

# Chargement de la matrice de transition
try:
    matrice_proba = np.loadtxt('transitions_alphabets.txt')
except Exception as e:
    raise FileNotFoundError("Erreur de chargement de la matrice de transition :", e)

# Vérification de la matrice
if matrice_proba.shape != (27, 27):
    raise ValueError("La matrice de transition n'a pas les dimensions attendues de 27x27.")

# Charger le message à déchiffrer
try:
    with open('MSG.txt', 'r') as f:
        message = list(f.read().strip().upper())
except Exception as e:
    raise FileNotFoundError("Erreur de chargement du message :", e)

# Vérification du message
alphabet = [' '] + [chr(i) for i in range(ord('A'), ord('Z') + 1)]
if not all(char in alphabet for char in message):
    raise ValueError("Le message contient des caractères non reconnus dans l'alphabet.")

print("Message à déchiffrer :", ''.join(message))

# Fonctions utilitaires
def dechiffrer_message_dict(message_code, alphabet, permutation_alphabet):
    decrypt_dict = {permutation_alphabet[i]: alphabet[i] for i in range(len(alphabet))}
    return [decrypt_dict[char] for char in message_code if char in decrypt_dict]

def calculer_V(texte, matrice_proba, alphabet, permutation_alphabet):
    texte_dechiffre = dechiffrer_message_dict(texte, alphabet, permutation_alphabet)
    V = 0
    for i in range(len(texte_dechiffre) - 1):
        c1, c2 = texte_dechiffre[i], texte_dechiffre[i + 1]
        idx1 = 0 if c1 == ' ' else alphabet.index(c1)
        idx2 = 0 if c2 == ' ' else alphabet.index(c2)
        V += np.log(matrice_proba[idx1][idx2] + 1e-15)
    return V / len(texte_dechiffre)

def recuit_simule_decrypt(message_code, matrice_proba, alphabet, max_iter, h, seed=1):
    npr.seed(seed)
    random.seed(seed)

    etat0 = random.sample(alphabet, len(alphabet))
    etat = etat0[:]
    val_etat = [etat]
    energie_vals = []

    for i in range(max_iter):
        etat_avant = etat[:]
        a, b = npr.choice(len(alphabet), 2, replace=False)
        etat[a], etat[b] = etat[b], etat[a]

        energie_etat = calculer_V(message_code, matrice_proba, alphabet, etat)
        energie_etat_avant = calculer_V(message_code, matrice_proba, alphabet, etat_avant)
        energie_vals.append(energie_etat)

        T = h / np.log(i + 2)
        if npr.rand() < min(1, np.exp((energie_etat - energie_etat_avant) / T)):
            val_etat.append(etat[:])
        else:
            etat = etat_avant[:]

        if energie_etat >= -2.12:
            print(f"Message décodé en {i+1} itérations.")
            break

    return val_etat, energie_vals

# Exécution multiple
max_iter = 10000
h = 0.016
iterations = 10
ens_energi = []
ens_mess_fin = []

for i in range(iterations):
    permutations, energie_vals = recuit_simule_decrypt(message, matrice_proba, alphabet, max_iter, h, seed=i)
    final_permutation = permutations[-1]
    final_energy = energie_vals[-1]
    decoded_message = ''.join(dechiffrer_message_dict(message, alphabet, final_permutation))
    print(f"Essai {i+1}: Message = {decoded_message}, Énergie finale = {final_energy:.4f}")
    ens_energi.append(final_energy)
    ens_mess_fin.append(decoded_message)

# Résultats globaux
print("\nRésumé des résultats :")
print(f"{'Énergie Finale':<20} {'Message Décodé'}")
print("-" * 80)
for energy, message in zip(ens_energi, ens_mess_fin):
    print(f"{energy:<20.4f} {message}")


In [ ]:

# Initialisation des listes pour stocker les énergies finales et les messages décodés
ens_energi = []
ens_mess_fin = []

# Paramètres pour le recuit simulé
max_iter = 10000  # Nombre maximal d'itérations
h = 0.013  # Température initiale

# Exécution de 10 essais indépendants
for i in range(10):
    # Exécution de l'algorithme de recuit simulé
    permutations, energie_vals = recuit_simule_decrypt(message, matrice_proba, alphabet, max_iter, h)
    
    # Dernière permutation trouvée et énergie finale
    final_permutation = permutations[-1]
    final_energy = energie_vals[-1]

    # Décodage du message avec la permutation finale
    decode_message = ''.join(dechiffrer_message_dict(message, alphabet, final_permutation))

    # Affichage des résultats pour chaque essai
    print(f"Essai {i+1}:")
    print(f"Message décodé : {decode_message}")
    print(f"Énergie finale : {final_energy}")
    print("\n")

    # Stockage des résultats dans les listes
    ens_energi.append(final_energy)
    ens_mess_fin.append(decode_message)

# Affichage global des résultats
#print("Résumé des énergies finales :", ens_energi)
#print("Résumé des messages décodés :", ens_mess_fin)


In [ ]:
# Affichage des énergies et des messages décodés en colonnes
print(f"{'Energie_final':<20} {'Message Décodé'}")
print("-"*80)  # Une ligne pour séparer les en-têtes des résultats

for msg, energy in zip(ens_energi,ens_mess_fin):
    print(f"{msg:<20}--> {energy}")
    